# Первичная настройка

## Настройка путей

In [6]:
import pandas as pd
from pathlib import Path


raw_dir  = Path("../data/raw")
images_dir = Path('../data/images')
preprocessed_path = Path("../data/preprocessed/text-and-image.csv")

## Проверка существоания

In [54]:
if not preprocessed_path.exists():
    main_df = pd.DataFrame(columns=["text", "image_name", "blocked"])
else: 
    main_df = pd.read_csv(preprocessed_path)
images_dir.mkdir(exist_ok=True)

# Загрузка данных

## neuralcatcher/hateful_memes
Все изображения загружаются локально из-за ограничения rate limit  
Ссылка: https://huggingface.co/datasets/neuralcatcher/hateful_memes

### Скачивание датасета

In [ ]:
from vk_mod.data import load_from_hugginface


max_rows = 5000
dataset_name = "neuralcatcher/hateful_memes"
ds_images_dir = Path('../data/raw/hateful_memes')
dataset_path = Path(raw_dir, dataset_name.split('/')[-1]).with_suffix(".csv")


load_from_hugginface(dataset_name, dataset_path, max_rows=max_rows)
temp_df = pd.read_csv(dataset_path)
temp_df.head(5)

Repo card metadata block was not found. Setting CardData to empty.


,Unnamed: 0,id,img,label,text
0,454,87049,img/87049.png,0,pauline hanson warns: obey our our rules our r...
1,773,10328,img/10328.png,1,how dikes look when they're in line paying for...
2,5989,16439,img/16439.png,1,cannibalism
3,2071,64320,img/64320.png,1,i said glass of juice not gas the jews
4,3704,4915,img/04915.png,0,they want this not beause they love you they n...


### Установка переводчика

In [9]:
import argostranslate
import argostranslate.package


def install_translation_model():
    argostranslate.package.update_package_index()
    available_packages = argostranslate.package.get_available_packages()
    package_to_install = next(
        filter(
            lambda x: x.from_code == "en" and x.to_code == "ru",
            available_packages
        )
    )
    argostranslate.package.install_from_path(package_to_install.download())

install_translation_model()

### Перевод текстов

In [10]:
import argostranslate.translate
import pandas as pd
from tqdm import tqdm


processed_data = []
for index, row in tqdm(temp_df.iterrows(), total=len(temp_df)):
    filename = row['img'].split('/')[-1]
    try:
        text_ru = argostranslate.translate.translate(
            row['text'], 
            from_code="en", 
            to_code="ru"
        )
        processed_data.append({
            'text': text_ru,
            'image_name': filename,
            'blocked': row['label']
        })
    except Exception as e:
        print(f"Ошибка перевода для строки {index}: {e}")

temp_df = pd.DataFrame(processed_data)
display(temp_df.sample(5))
display(temp_df.loc[:, temp_df.columns == "blocked"].value_counts().reset_index(name="count"))

100%|██████████| 5000/5000 [36:26<00:00,  2.29it/s]  


,text,image_name,blocked
4834,"Чувак, прекрати курить, это навредит тебе.",19427.png,0
3429,"Слышу ли я флейты, барабаны и саш",59047.png,0
4436,"А потом Трамп сказал, что я действительно умны...",09657.png,0
3257,"Я собиралась пошутить над мусульманами, но они...",95763.png,1
4981,Если жизнь черных действительно имеет значение...,72084.png,1


,blocked,count
0,0,3228
1,1,1772


### Перенос изображений

In [32]:
import shutil
from tqdm import tqdm

cleaned_data = []
for index, row in tqdm(temp_df.iterrows(), total=len(temp_df)):
    initial_path = Path(ds_images_dir, row["image_name"])
    target_path = Path(images_dir, row["image_name"])
    try:
        shutil.copy(initial_path, target_path)
        cleaned_data.append({
            'text': row["text"],
            'image_name': row["image_name"],
            'blocked': row["blocked"]
        })
    except Exception as e:
        pass
temp_df = pd.DataFrame(cleaned_data)
display(temp_df.sample(5))
display(temp_df.loc[:, temp_df.columns == "blocked"].value_counts().reset_index(name="count"))

100%|██████████| 5000/5000 [00:08<00:00, 594.84it/s]


,text,image_name,blocked
2659,Когда вы случайно желаете «сладких снов» своем...,23174.png,1
1829,кот Брюса Дженнера,70356.png,1
1330,"Я положил ""обезьяну"" в изнасилование",49807.png,0
872,Хороший день для селфи,94731.png,0
2062,Хватит дурачиться.,80927.png,0


,blocked,count
0,0,2538
1,1,1404


### Обьединение с основным датасетом

In [33]:
main_df = pd.concat([main_df, temp_df])
display(main_df.sample(5))

,Unnamed: 0,text,image_name,blocked
38212,29712.0,Этот ниггер тупой https://t.co/OGvw5...,1115786429974700033.jpg,0
7967,7967.0,"Я буду ненавидеть то, как я говорю, я гарантир...",72805.png,0
57925,49425.0,Познакомьтесь с новейшим теоретиком заговора в...,1045032938243534848.jpg,0
29233,20733.0,Припаркованная сука нигга https://t.co/IpxtFknrQH,1113149506659512320.jpg,0
53219,44719.0,@EmonieMonaee 5’8... но Лил Нигга заставил мен...,1114256906124767232.jpg,0


## victorcallejasf/multimodal-hate-speech
Датасет загружается локально из-за размера  
Ссылка: https://www.kaggle.com/datasets/victorcallejasf/multimodal-hate-speech

### Скачивание датасета

In [38]:
import json
from pathlib import Path


dataset_path = Path("../data/raw/MMHS150K_GT.json")
ds_images_dir = Path("../data/raw/MMHS150K/img_resized")
with open(dataset_path, 'r', encoding='utf-8') as f:
    data_dict = json.load(f)

### Перевод текстов

In [42]:
import argostranslate.translate
from tqdm import tqdm
from itertools import islice

max_items = 27000
procesesed_data = []

for index, row in tqdm(islice(data_dict.items(), max_items), total=max_items):
    text = argostranslate.translate.translate(
                                        row['tweet_text'],
                                        from_code="en",
                                        to_code="ru"
                                        )
    labels = [1 for label in row['labels'] if label != 0]
    label = 1 if sum(labels) >= 2 else 0
    filename = f"{index}.jpg"
    procesesed_data.append({
        "image_name": filename,
        "text": text,
        "blocked": label
    })
temp_df = pd.DataFrame(procesesed_data)
display(temp_df.sample(5))
display(temp_df.loc[:, temp_df.columns == "blocked"].value_counts().reset_index(name="count"))

  0%|          | 0/27000 [00:00<?, ?it/s]

100%|██████████| 27000/27000 [6:04:46<00:00,  1.23it/s]   


,image_name,text,blocked
2082,1105565344347770880.jpg,Когда один нигга стоит в турни https://t.co/RC...,0
5728,1110268253153255424.jpg,Молодой нигга Блиссссед! https://t.co/hlAzKUdBmq,0
26401,1054515930514878464.jpg,@tetsupay @tsubasa_yozawa Высококачественный t...,0
4401,1063734190842081280.jpg,"Я только что поняла, что Элмер Фадд - чертова ...",1
16630,1116078230761558016.jpg,Юртта мир Чиханда мир мой ниггер https://t.co/...,0


,blocked,count
0,0,19258
1,1,7742


### Перенос изображений

In [44]:
import shutil
from tqdm import tqdm


cleaned_data = []
for index, row in tqdm(temp_df.iterrows(), total=len(temp_df)):
    initial_path = Path(ds_images_dir, row["image_name"])
    target_path = Path(images_dir, row["image_name"])
    try:
        shutil.copy(initial_path, target_path)
        cleaned_data.append({
            'text': row["text"],
            'image_name': row["image_name"],
            'blocked': row["blocked"]
        })
    except Exception as e:
        pass
temp_df = pd.DataFrame(cleaned_data)
display(temp_df.sample(5))
display(temp_df.loc[:, temp_df.columns == "blocked"].value_counts().reset_index(name="count"))

100%|██████████| 27000/27000 [01:13<00:00, 368.67it/s]


,text,image_name,blocked
4921,Вы можете спорить со своими родителями Buh cas...,1114185027040968704.jpg,1
10004,RT me if you think I'm a pathetic faggot https...,1105493926780248070.jpg,1
12171,"Этот ниггер думал, что это фотография. https:/...",1113974771967041536.jpg,1
25292,"Они любят меня, потому что я холодный, и вы не...",1114725029420457984.jpg,0
18940,Сдаться сегодня? https://t.co/6t4nqzpMF1,1108091721005436928.jpg,1


,blocked,count
0,0,19258
1,1,7742


In [55]:
main_df = pd.concat([main_df, temp_df])
display(main_df)

,text,image_name,blocked
0,@FriskDontMiss Нигга https://t.co/cAsaLWEpue,1114679353714016256.jpg,1
1,Мои лошади отсталые https://t.co/HYhqc6d5WN,1063020048816660480.jpg,1
2,"""NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...",1108927368075374593.jpg,0
3,RT xxSuGVNGxx: Я наткнулся на эту СВЕРХОВУЮ НЬ...,1114558534635618305.jpg,0
4,Все зовут тебя Ниггер! https://t.co/6mguJ6KIBF,1035252480215592966.jpg,1
...,...,...,...
26995,#HEEJIN: права дамб! https://t.co/YPh16vwbkv,1115362312582516738.jpg,0
26996,Wtf my nigga #PS4share https://t.co/QeUYQYASZy...,1107008893304127493.jpg,0
26997,@MenaaSzn @SaeddyJ Lmaoooooooooo nigga said ru...,1108918680795131906.jpg,0
26998,"@CNN Я знал, что мы должны бояться того, что о...",1037251470847029248.jpg,1


# Final

## Clean

In [56]:
main_df = (main_df
           .drop("Unnamed: 0", axis=1, errors="ignore")           
           .drop_duplicates()
           .dropna(subset=["text", "image_name", "blocked"]))
main_df['blocked'] = main_df['blocked'].astype(int)

In [57]:
display(main_df.loc[:, main_df.columns == "blocked"].value_counts().reset_index(name="count"))
print(main_df.shape)

,blocked,count
0,0,19258
1,1,7742


(27000, 3)


## Save

In [58]:
preprocessed_path.parent.mkdir(exist_ok=True)
main_df.to_csv(preprocessed_path, index=False)